# Quickstart tutorial: Enter the _Vivarium_

First follow the installations instruction in the README of the repository. If everything works correctly, you should have launched the web interface.

## Introduction to the web interface

For the web interface tutorial, please refer to [this file.](web_interface_tutorial.md)

## Controlling the simulator programmatically

Instead of controlling the simulator from the web interface, you can also control it programmatically, e.g. directly from this Jupyter notebook. First make sure that the simulator is stopped in the web interface. Then execute:

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()

The newly created `controller` object provides all the functionalities to easily interact with the simulator server. Let's look for example at the agents present in the environment:

In [2]:
controller.agents

This is a list of `Agent` objects, one per agent in the environment. Let's for example access the configuration of the first agent in this list and look at its attribute using the Python `.__dict__` syntax:

In [3]:
controller.agents[0].__dict__

{'config': AgentConfig(behavior='manual', color='#ffff00', diameter=3.0, exists=True, friction=0.10000000149011612, idx=0, left_motor=0, left_prox=0.6139006614685059, mass_center=1.0, mass_orientation=0.125, name='AgentConfig00004', orientation=-194.20037841796875, proxs_cos_min=0.0, proxs_dist_max=100.0, right_motor=0, right_prox=0.7286465764045715, speed_mul=1.0, theta_mul=1.0, wheel_diameter=2.0, x_position=93.14191436767578, y_position=72.39128112792969),
 '_routines': {},
 'user_events': {},
 'etype': <EntityType.AGENT: 0>,
 'behaviors': {}}

### Modifying agents

We will now see how to interact with agents by modifying their attributes. First, we need to start the simulation in this notebook with the following instruction:

In [4]:
# The `threaded=True` argument ensure that this cell does not block the notebook
controller.run(threaded=True)

We can stop the simulation at any time by executing the following instruction:

In [5]:
controller.stop()

However, this will cause the changes made in this notebook to no longer reflect on the interface, so we should start it again:

In [6]:
controller.run(threaded=True)

We saw in the dictionary the value of different attributes of the agent, for example its position, diameter or color. Let's make the first agent (index 0) larger by increasing its `diameter` attribute:

In [7]:
controller.agents[0].diameter = 10

If you now look at the web interface, you should see that one of the agent is now twice larger than the others. All changes made in this notebook will be automatically visible in the web interface if the server is running, and vice versa. One can for example also change the color of an agent, let's do it on the third agent in the list now (i.e. the agent with index 2):

In [8]:
controller.agents[0].color = 'yellow'

The same principles apply to objects, which are accessible through `controller.objects`.

### Controlling the agent's movement

We can modify the appearance of the agents, but we can also decide how they move. For now, let's set the speed of the left wheel of the first agent, to 1 radian per second:

In [9]:
controller.agents[0].left_motor = 1.

You should now see the agent spinning. To make it move in a straight line, we need to set the other wheel to the same speed, with the following:

In [10]:
controller.agents[0].right_motor= 1.

We can reset the motors of the agents with this instruction:

In [12]:
controller.agents[0].left_motor = controller.agents[0].right_motor = 0.

### Retrieving the proximeters values

The agents also come with a set of sensors called `proximeters`, that activate when the agent approaches another entity.

We can retrieve the value of these sensors with this instruction:

In [13]:
print(f"right prox: {controller.agents[0].right_prox}")
print(f"left prox: {controller.agents[0].left_prox}")

right prox: 0.6962598562240601
left prox: 0.9220200181007385
